In [11]:
import duckdb
import pandas as pd

# Connect to the database
conn = duckdb.connect("database.db")
c = conn.cursor()

In [12]:
# Step 1: Get distinct species with egg data
species_query = """
SELECT Code, Scientific_name
FROM Species s
INNER JOIN Bird_nests n ON s.Code = n.Species
INNER JOIN Bird_eggs e ON n.Nest_ID = e.Nest_ID
GROUP BY s.Code, s.Scientific_name
ORDER BY s.Scientific_name;
"""

In [13]:
# Step 2: Iterate over species
for row in c.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]

    # Step 3: Query egg data for the current species
    egg_query = """
    SELECT Width, Length
    FROM Bird_eggs
    WHERE Nest_ID IN (
        SELECT Nest_ID
        FROM Bird_nests
        WHERE Species = ?
    );
    """
    df = pd.read_sql(egg_query, conn, params=[species_code])

    # Step 4: Compute statistics and show results
    if not df.empty:
        df['Volume'] = df['Width'] ** 2 * df['Length']
        cv = round(df['Volume'].std() / df['Volume'].mean() * 100, 2)
        print(f"{scientific_name} {cv}%")
    else:
        print(f"{scientific_name} No egg data available")

# Close the connection
conn.close()

Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%


<ipython-input-13-68ec66bdcea7>:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=[species_code])
<ipython-input-13-68ec66bdcea7>:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=[species_code])
<ipython-input-13-68ec66bdcea7>:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=[species_code])
<ipython-input-13-68ec66bdcea7>:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection)